# Collecting Digital Trace Data: Web Scraping / APIs
June 19th, 2018 - Javier Garcia-Bernardo & Allie Morgan

In [1]:
## Requirements
import requests               # Simple HTTP operations (GET and POST)
import selenium               # Loads dynamic (javascript) pages
import json                   # Parsing the responses from APIs
import re                     # Python library for parsing regular expressions
from bs4 import BeautifulSoup # Parsing HTML

## Web Scraping

[Web scraping](https://en.wikipedia.org/wiki/Web_scraping) is a method for extracting data from the web. There are many techniques which can be used for web scraping — ranging from requiring human involvement (“human copy-paste”) to fully automated systems (using computer vision). Somewhere in the middle is HTML parsing, which we will describe here.

Web scraping using [HTML parsing](https://en.wikipedia.org/wiki/Web_scraping#HTML_parsing) is often used on webpages which share similar HTML structure. For example, you might want to scrape the ingredients from chocolate chip cookie recipes to identify correlations between ingredients and five-star worthy cookies, or you might want to predict who will win March Madness by looking at game play-by-plays, or you want to know all the local pets up for adoption.

### Static Webpages

In [2]:
pet_pages = ["https://www.boulderhumane.org/animals/adoption/dogs", 
             "https://www.boulderhumane.org/animals/adoption/cats", 
             "https://www.boulderhumane.org/animals/adoption/adopt_other"]

r = requests.get(pet_pages[0])
html = r.text
print(html[:500]) # Print the first 500 characters of the HTML

<!DOCTYPE html>
<head>
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta charset="utf-8" />
<link rel="shortcut icon" href="https://www.boulderhumane.org/sites/default/files/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="Generator" content="Drupal 7 (http://drupal.org)" />
<meta name="viewport" content="width=1000px, initial-scale=1.0, maximum-scale=1.0" />
<title>Dogs Available for Adoption | Humane Society of Boulder Valley</title>
<link type="text/css" rel="stylesheet


When you visit a webpage, your web browser renders an HTML document with CSS and Javascript to produce a visually appealing page. (See the HTML above.) [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) is a Python library for parsing HTML. We'll use it to extract all of the names, ages, and breeds of the [dogs](https://www.boulderhumane.org/animals/adoption/dogs), [cats](https://www.boulderhumane.org/animals/adoption/cats), and [small animals](https://www.boulderhumane.org/animals/adoption/adopt_other) currently up for adoption at the Boulder Humane Society.

In [3]:
soup = BeautifulSoup(html, 'html.parser')

Note, that the feature of these pages which we are exploiting is their repeated HTML structure. Every animal listed has the following HTML variant:
```{html}
<div class="views-row ... ">
  ...
  <div class="views-field views-field-field-pp-animalname">
    <div class="field-content">
      <a href="/animals/adoption/" title="Adopt Me!">Romeo</a>
    </div>
  </div>
  <div class="views-field views-field-field-pp-primarybreed">
    <div class="field-content">New Zealand</div>
  </div>
  <div class="views-field views-field-field-pp-secondarybreed">
    <div class="field-content">Rabbit</div>
  </div>
  <div class="views-field views-field-field-pp-age">
    ...
    <span class="field-content">0 years 2 months</span>
  </div>
  <div class="views-field views-field-field-pp-gender">
    ...
    <span class="field-content">Male</span>
  </div>
  ...
</div>
``` 
So to get at the HTML object for each pet, we can run the following:

In [4]:
pets = soup.find_all('div', {'class': re.compile('.*views-row.*')})

That is, find all of the `div` tags with the `class` attribute which contains the string `views-row`. 

Next to grab the name, breeds, and ages of these pets, we’ll grab the children of each pet HTML object. For example:

In [5]:
head = "views-field views-field-field-pp-"
for pet in pets:
    name = pet.find('div', {'class': head + 'animalname'}).get_text(strip=True)
    primary_breed = pet.find('div', {'class': head + 'primarybreed'}).get_text(strip=True)
    secondary_breed = pet.find('div', {'class': head + 'secondarybreed'}).get_text(strip=True)
    age = pet.find('div', {'class': head + 'age'}).get_text(strip=True)
    print(name, primary_breed, secondary_breed, age)

(u'Audi', u'Belgian Malinois', u'', u'Age:5 years 7 months')
(u'Kobe', u'Cane Corso', u'', u'Age:1 year 8 months')
(u'Roxy', u'Terrier, American Pit Bull', u'Mix', u'Age:1 year 6 months')
(u'Bear', u'Retriever, Chesapeake Bay', u'Mix', u'Age:2 years 5 months')
(u'Chloe', u'Akita', u'Mix', u'Age:5 years 0 months')
(u'Drew', u'Retriever, Labrador', u'Retriever, Golden', u'Age:2 years 0 months')
(u'Harley', u'Terrier, American Pit Bull', u'Mix', u'Age:2 years 0 months')
(u'Sadie', u'Great Dane', u'Retriever, Labrador', u'Age:8 years 10 months')
(u'Megrita', u'Dutch Shepherd', u'Mix', u'Age:5 years 0 months')
(u'Molly', u'Mastiff', u'Rottweiler', u'Age:8 years 0 months')
(u'Butterbean', u'Terrier, Jack Russell', u'Mix', u'Age:7 years 0 months')
(u'Rowdy', u'Rottweiler', u'Mix', u'Age:6 years 0 months')
(u'Marco', u'Spaniel, American Cocker', u'', u'Age:5 years 0 months')
(u'Eva', u'Boxer', u'Mix', u'Age:6 years 0 months')
(u'Stew', u'Chihuahua, Short Coat', u'Mix', u'Age:7 years 0 months')

where each call to `find` is getting the children of a pet object, in particular, the `div`s with `class` attributes which look like `views-field views-field-field-pp-*`. Feel free to replace the above code with the cat or small animal pages provided and see how the output changes.

### Dynamic (Javascript) Webpages

Above, we requested webpages that required no [Javascript](https://en.wikipedia.org/wiki/JavaScript). In other words, there was no input required on the users' end to view the content of the page (e.g. a login). Let's try a more complicated example of webscraping where content is loaded dynamically.

Some characteristics of HTML scraping with [Selenium](https://www.seleniumhq.org/download/) it: (b) can handle javascript, (c) get **HTML** back after the Javascript has been rendered, (d) can behave like a person, though it (a) can be slow. 

Requirements (one of the below):
- Firefox + geckodriver (https://github.com/mozilla/geckodriver/releases)
- Chrome + chromedriver (https://sites.google.com/a/chromium.org/chromedriver/)
    
Note: geckodriver/chromedriver must have execution permissions (chmod +x geckodriver)

In [20]:
import selenium.webdriver

Start the browser and define how much are you willing to wait for a page to load. (Many times this is not needed but it doesn't hurt.)

In [36]:
# Open the driver (change the executable path to geckodriver_mac.exe or geckodriver.exe)
# driver = selenium.webdriver.Chrome(executable_path="./chromedriver")
driver = selenium.webdriver.Chrome()
driver.implicitly_wait(10)
driver.set_page_load_timeout(30)

Visit [xkcd](https://xkcd.com) and click through the comics.

In [25]:
# Get the xkcd website
driver.get("https://xkcd.com/")

In [26]:
# Let's find the 'random' buttom
element = driver.find_element_by_xpath('//*[@id="middleContainer"]/ul[1]/li[3]/a')
element.click()

Find an attribute of this page.

In [27]:
element = driver.find_element_by_xpath('//*[@id="comic"]/img')
element.get_attribute("title")

u"Grayton also proposed making college scholarships available exclusively to sexually active teens, amnesty for illegal immigrants who create room for themselves by killing a citizen, and a graduated income tax based on penis size. He has been endorsed by Tracy Morgan, John Wilkes Booth's ghost, and the Time Cube guy."

Let's visit a webpage which requires a login. Signing in to Facebook ...

In [32]:
##DO NOT WRITE YOUR PASSWORD IN NOTEBOOKS!!
fb_email, fb_pass = "f1692418@mvrht.com","pedropalotes"

In [37]:
# Go to Facebook
driver.get("https://www.facebook.com/")

In [38]:
# Send email and password
driver.find_element_by_xpath('//*[@id="email"]').send_keys(fb_email)
driver.find_element_by_xpath('//*[@id="pass"]').send_keys(fb_pass)

WebDriverException: Message: unknown error: call function result missing 'value'
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.33.506106 (8a06c39c4582fbfbab6966dbb1c38a9173bfb1a2),platform=Mac OS X 10.13.5 x86_64)


In [ ]:
# Click on login
driver.find_element_by_xpath('//*[@id="loginbutton"]').click()

In [88]:
# Find JP
element = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div/div/div/div[1]/div[2]/div/form/div/div/div/div/input[2]')
element.send_keys("john paul gonzales")
element = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/div/div/div/div[1]/div[2]/div/form/button')
element.click()

In [89]:
# Click on him
element = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div/div[3]/div[2]/div/div/div[3]/div/div/div/div[1]/div/div/div/div/div/div[2]/div[1]/div/div[1]/a/div')
element.click()

In [ ]:
# Send a friend request
element = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div/div[4]/div/div[2]/div/div[2]/span/span/span[1]/a')
element.click()

In [39]:
# Always remember to close your browser!
driver.close()

## APIs

To allow users to access large amounts of data, companies may provide an [Application Programming Interface (API)](https://en.wikipedia.org/wiki/Application_programming_interface). Often these request are handled via PUT and POST HTTP requests. For example, to make a request from the Twitter API:

```{bash}
curl --request GET 
 --url 'https://api.twitter.com/1.1/search/tweets.json?q=nasa&result_type=popular' 
 --header 'authorization: OAuth oauth_consumer_key="consumer-key-for-app", ... , 
 oauth_token="access-token-for-authed-user", oauth_version="1.0"'
 ```

APIs often return data in the format of [Javascript Object Notation (JSON)](https://en.wikipedia.org/wiki/JSON). For example:

```{json}
{"status": 200, "message": "hello world"}
```

### "Hidden" APIs

First, let's try and access what we are calling a "hidden" API. That is, we investigate the resources requested by a webpage (e.g. a list of faculty), and make requests directly to that API.

In [53]:
import requests
import json

def get_names(letters):
    params = (
        ('name', letters),
        ('request_num', '1'),
    )

    response = requests.get('https://www.uvm.edu/directory/api/query_results.php', params=params)
    if response.ok == True:
        return response.text
    else:
        return None

In [54]:
response = get_names("ab")

In [55]:
response_json = json.loads(response)
response_json

{u'authenticated': u'',
 u'data': [{u'0': u'uvmedubroadcastflag',
   u'1': u'uvmeduuuid',
   u'10': u'edupersonprimaryaffiliation',
   u'11': u'uvmeduprimaryaffiliation',
   u'12': u'ou',
   u'13': u'title',
   u'14': u'cn',
   u'15': u'givenname',
   u'16': u'postaladdress',
   u'17': u'mailroutingaddress',
   u'18': u'uid',
   u'19': u'uidnumber',
   u'2': u'edupersonaffiliation',
   u'20': u'gidnumber',
   u'21': u'gecos',
   u'22': u'homedirectory',
   u'23': u'loginshell',
   u'24': u'edupersonprincipalname',
   u'25': u'mail',
   u'26': u'uvmedualias',
   u'27': u'telephonenumber',
   u'3': u'uvmeduaffiliation',
   u'4': u'sn',
   u'5': u'uvmedusurname',
   u'6': u'uvmaltuid',
   u'7': u'uvmaltgid',
   u'8': u'uvmalthomedir',
   u'9': u'edupersonentitlement',
   u'cn': {u'0': u'Abraham Isaac Sender', u'count': u'1'},
   u'count': u'28',
   u'dn': u'uid=asender,ou=People,dc=uvm,dc=edu',
   u'edupersonaffiliation': {u'0': u'Alum',
    u'1': u'Affiliate',
    u'count': u'2'},
   u'e

In [56]:
for i, person in enumerate(response_json["data"]):
    if i == 10: 
        break # Make sure we don't print too much
        
    print(person["edupersonprimaryaffiliation"]["0"], person["edupersonprincipalname"]["0"], person["cn"]["0"])

(u'Affiliate', u'asender@uvm.edu', u'Abraham Isaac Sender')
(u'Faculty', u'acrocker@uvm.edu', u'Abigail Miles Crocker')
(u'Faculty', u'aambaye@uvm.edu', u'Abiy B. Ambaye')
(u'Staff', u'aldattil@uvm.edu', u'Abbey L. Peterson')
(u'Faculty', u'avan@uvm.edu', u'Abby Katrien van den Berg')
(u'Faculty', u'amcgowan@uvm.edu', u'Abigail S. McGowan')
(u'Student', u'afsmith@uvm.edu', u'Abigayle Frances Smith')
(u'Student', u'alangsne@uvm.edu', u'Abigail Elizabeth Langsner')
(u'Faculty', u'atangada@uvm.edu', u'Abhilasha Tangada')
(u'Faculty', u'awoodhea@uvm.edu', u'Abigail Rhodes Adler')


### Explicit APIs

Next, let's try a more typical example of an API. The perks of this approach: (a) send a request and get back JSON, (b) chances are that somebody else has created a Python wrapper for you, but keep in mind that (c) APIs have limits.

Let's consider a common API example -- Twitter. To get started:
- Get a key: https://apps.twitter.com/
- Documentation: https://dev.twitter.com/rest/public
- Find a library: https://dev.twitter.com/resources/twitter-libraries (We'll use https://github.com/tweepy/tweepy)

Limitations: 100 messages / query, 180 messages every 15 min, & only the last seven days of data 

In [57]:
!pip install tweepy

In [59]:
import tweepy
import time
import pickle

def twitter(d_keys,query):
    #Authtentify
    auth = tweepy.OAuthHandler(d_keys["CONSUMER_KEY"], d_keys["CONSUMER_SECRET"])
    auth.set_access_token(d_keys["ACCESS_KEY"], d_keys["ACCESS_SECRET"])
    api = tweepy.API(auth)

    # We want 1000 tweets
    num_results = 1000
    result_count = 0
    last_id = None
    
    # Max 180 tweets 15 min
    cumulative = 0

    #While we don't have them
    while (result_count <  num_results):
        previous_tweets = result_count
        # Ask for more tweets, starting in the 'last_id' (identifier of the tweet)
        results = api.search(q = query,
                              count = 90, max_id = last_id, result_type="recent")
                                # geocode = "{},{},{}km".format(latitude, longitude, max_range) #for geocode

        # For each tweet extract some info (JSON structure)
        for result in results:
            result_count += 1
            user = result.user.screen_name
            text = result.text
            followers_count = result.user.followers_count
            time_zone = result.user.time_zone
            print("_"*10)
            print(user,time_zone,followers_count)
            print(text)

        # Keep the last_id to know where to continue
        last_id = int(result.id)-1
        new_tweets = result_count - previous_tweets

        print ("Number of results: {} ({} new)".format(result_count,new_tweets))

        # If we don't get new tweets exit
        if new_tweets == 0: 
            break
        
        time.sleep(1)
        
        if ((result_count + 90) // 150) > cumulative:
            cumulative += 1
            time.sleep(15*60)


d_keys = pickle.load(open(".key","rb")) # Don't share your keys ;)
twitter(d_keys,"from:sfiscience")

IOError: [Errno 2] No such file or directory: '.key'